In [1]:
#matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from dataset.dataloader import *
import operators as ops
import os

CKPT_DIR = "./logs/checkpoints"

In [2]:
class VGG8:
    def __init__(self):
        self.criterion = ops.SoftmaxWithLoss()
        self.conv_layers = [
            # Layer 1 (B, 1, 28, 28) -> (B, 32, 28, 28)
            ops.Conv2D(in_channels=1, out_channels=32, kernel_size=3, padding=1, stride=1, name="L1_C"),
            ops.ReLU(name="L1_R"),

            # Layer2 (B, 32, 28, 28) -> (B, 64, 14, 14)
            ops.Conv2D(in_channels=32, out_channels=64, kernel_size=3, padding=1, stride=1, name="L2_C"),
            ops.ReLU(name="L2_R"),
            ops.MaxPooling(kernel_size=2, stride=2, name="L2_M"),
            
            # Layer 3 (B, 64, 14, 14) -> (B, 64, 14, 14)
            ops.Conv2D(in_channels=64, out_channels=64, kernel_size=3, padding=1, stride=1, name="L3_C"),
            ops.ReLU(name="L3_R"),

            # Layer 4 (B, 64, 14, 14) -> (B, 128, 7, 7)
            ops.Conv2D(in_channels=64, out_channels=128, kernel_size=3, padding=1, stride=1, name="L4_C"),
            ops.ReLU(name="L4_R"),
            ops.MaxPooling(kernel_size=2, stride=2, name="L4_M"),

            # Layer 5 (B, 128, 7, 7) -> (B, 256, 7, 7)
            ops.Conv2D(in_channels=128, out_channels=256, kernel_size=3, padding=1, stride=1, name="L5_C"),
            ops.ReLU(name="L5_R"),

            # Layer 6 (B, 256, 7, 7) -> (B, 256, 7, 7)
            ops.Conv2D(in_channels=256, out_channels=256, kernel_size=3, padding=1, stride=1, name="L6_C"),
            ops.ReLU(name="L6_R")
        ]

        # Layer 7 (B, 256*7*7) -> (B, 256)
        self.fc_layers = [
            ops.FullyConnected(in_feature=256*7*7, out_feature=256, name="L7_FC"),
            ops.ReLU(name="L7_R"),

        # Layer 8 (B, 256) -> (B, 10)
            ops.FullyConnected(in_feature=256, out_feature=10, name="L8_FC")
        ]

    def backprop(self, pred: np.ndarray, label: np.ndarray, lr) -> None:
        # Backward
        #dout = self.criterion.backward(pred, label)
        dout = self.criterion.backward(1)
        print(dout)
        for i in range(len(self.fc_layers)-1, -1, -1):
            dout = self.fc_layers[i].backward(dout)
        dout = dout.reshape(dout.shape[0], 256, 7, 7)
        for i in range(len(self.conv_layers)-1, -1, -1):
            dout = self.conv_layers[i].backward(dout)
        # Update
        for layer in self.conv_layers:
            layer.update(lr)
        for layer in self.fc_layers:
            layer.update(lr)
        return dout

    def forward(self, x: np.ndarray):
        i = 0
        for layer in self.conv_layers:
            i+=1
            x = layer.forward(x) 
        x = x.reshape(x.shape[0],-1)
        for layer in self.fc_layers:
            x = layer.forward(x)
        return x

    def save(self, fileName: str):
        with open(fileName, "wb") as f:
            pickle.dump(self, f)

    def load(self, fileName: str):
        with open(fileName, "rb") as f:
            self = pickle.load(f)

In [3]:
model = VGG8()

In [4]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

batch_size = 2
learning_rate = 0.01
loss_print_time = batch_size * 30
x_log = []
loss_log = []

print(f"Train:{x_train.shape}, Test:{x_test.shape}")

Train:(60000, 784), Test:(10000, 784)


In [5]:

for epoch in range(10):
    accuracy = 0.0
    train_size = x_train.shape[0]
    running_loss = 0.0
    for i in tqdm(range(0, train_size, batch_size)):
        x = x_train[i:i+batch_size].reshape(batch_size, 1, 28, 28)   # B, C, H, W
        labels = t_train[i:i+batch_size]
        #optimizer.zero_grad()

        outputs = ops.Softmax(model.forward(x))
        #print(outputs)
        labels = labels
        loss = model.criterion.forward(outputs, labels)
        running_loss += loss
        model.backprop(outputs, labels, learning_rate)

        if (i % loss_print_time == 0):
            #running_loss /= loss_print_time
            print(f'[epoch:{epoch}, data num:{i}] loss: {running_loss}')
            x_log.append(i)
            loss_log.append(running_loss)
            running_loss = 0.0
            #model.save(os.path.join(CKPT_DIR, "epoch_%03d_%06d.pkl" % (epoch, i) ))




  0%|          | 0/30000 [00:00<?, ?it/s]

[[ 2.42312874e-06  4.62385159e-02  2.98390819e-01  3.45544050e-12
   6.52586248e-16 -4.67916509e-01  3.56429486e-11  4.12006416e-19
   9.51330459e-24  1.23284750e-01]
 [-4.99999218e-01  3.95233269e-02  3.12772223e-01  2.74925001e-13
   2.19705838e-17  2.72165518e-02  3.69271441e-12  6.51714088e-21
   5.03042690e-26  1.20487116e-01]]
[epoch:0, data num:0] loss: 8.057041432086374
[[ 0.05058741  0.04990148  0.04930703  0.04999886 -0.4500012   0.05048789
   0.04999887  0.04999885  0.04999881  0.04972201]
 [ 0.05058741 -0.45009852  0.04930703  0.04999886  0.0499988   0.05048789
   0.04999887  0.04999885  0.04999881  0.04972201]]


  0%|          | 2/30000 [00:00<30:05, 16.62it/s]

[[ 0.05019951  0.05015717  0.04964607  0.04994973  0.0502001   0.05016985
   0.04994973  0.04994973  0.04994973 -0.45017161]
 [ 0.05019951  0.05015717 -0.45035393  0.04994973  0.0502001   0.05016985
   0.04994973  0.04994973  0.04994973  0.04982839]]
[[ 0.0501493  -0.44989296  0.04984557  0.04990002  0.05014989  0.0501197
   0.04990002  0.04990002  0.04990002  0.05002844]
 [ 0.0501493   0.05010704  0.04984557 -0.45009998  0.05014989  0.0501197
   0.04990002  0.04990002  0.04990002  0.05002844]]


  0%|          | 5/30000 [00:00<28:03, 17.82it/s]

...

  2%|▏         | 451/30000 [00:21<23:05, 21.32it/s]

[[ 0.04941657  0.05298934  0.05062705  0.04902585 -0.44943902  0.04834981
   0.04856152  0.05170777  0.04846433  0.05029677]
 [ 0.04941657  0.05298934  0.05062705 -0.45097415  0.05056098  0.04834981
   0.04856152  0.05170777  0.04846433  0.05029677]]
[[ 0.04936791  0.05293337  0.05057596  0.04922345 -0.44923679  0.04830322
   0.04851452  0.05165448  0.04841752  0.05024635]
 [ 0.04936791  0.05293337  0.05057596  0.04922345 -0.44923679  0.04830322
   0.04851452  0.05165448  0.04841752  0.05024635]]
[[ 0.04931825  0.05287636  0.05052387  0.04917408  0.05122039  0.04825567
   0.04846655 -0.44839983  0.04836974  0.05019493]
 [ 0.04931825  0.05287636  0.05052387  0.04917408  0.05122039 -0.45174433
   0.04846655  0.05160017  0.04836974  0.05019493]]
[[ 0.04926964  0.05282049  0.05047286  0.04912576  0.05116796  0.04845078
   0.04841961  0.05180534  0.04832299 -0.44985542]
 [ 0.04926964  0.05282049  0.05047286  0.04912576  0.05116796  0.04845078
  -0.45158039  0.05180534  0.04832299  0.0501445

  2%|▏         | 455/30000 [00:21<23:42, 20.77it/s]


[[ 0.04941998  0.05270993  0.05037194  0.04903017  0.05106423  0.04835781
   0.04856876 -0.448301    0.04823051  0.05054768]
 [ 0.04941998 -0.44729007  0.05037194  0.04903017  0.05106423  0.04835781
   0.04856876  0.051699    0.04823051  0.05054768]]
[[ 0.04936892  0.05291593  0.05031894  0.04897989 -0.4489902   0.04830888
   0.04851941  0.0519021   0.04818183  0.05049432]
 [ 0.04936892  0.05291593 -0.44968106  0.04897989  0.0510098   0.04830888
   0.04851941  0.0519021   0.04818183  0.05049432]]
[[ 0.04931949  0.05285919  0.05051957  0.04893123  0.05121248  0.04826153
   0.04847165 -0.4481525   0.04813473  0.05044262]
 [ 0.04931949  0.05285919  0.05051957 -0.45106877  0.05121248  0.04826153
   0.04847165  0.0518475   0.04813473  0.05044262]]
[[ 0.04927043  0.05280288  0.05046811  0.04912797  0.0511596   0.04821454
  -0.45157574  0.05205293  0.04808798  0.05039132]
 [ 0.04927043  0.05280288  0.05046811  0.04912797  0.0511596   0.04821454
   0.04842426 -0.44794707  0.04808798  0.0503913

KeyboardInterrupt: 

In [ ]:
plt.plot(x_log, loss_log)
plt.show()

In [ ]:
print(t_train.shape)
batch_size = 200
total_correct = 0

running_loss = 0.0
accuracy = 0.0
test_size = 1000
test_data_idx = np.random.choice(x_test.shape[0], test_size)
for i in tqdm(test_data_idx):
    x = x_test[i].reshape(1, 1, 28, 28)
    labels = t_test[i]

    outputs = model.forward(x)
    labels = labels.reshape(outputs.shape)
    c = (np.argmax(labels, 1) == np.argmax(outputs, 1)).squeeze()
    total_correct += np.sum(c)

print(f"Accuracy:{total_correct/test_size}")
